In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

# Load the test image
image = cv2.imread('Lena.jpg')

# Convert the image to RGB (OpenCV loads images in BGR format)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Apply SLIC superpixel segmentation
num_segments = 100  # You can adjust this parameter based on your preference
slic = cv2.ximgproc.createSuperpixelSLIC(image, algorithm=cv2.ximgproc.SLIC, region_size=10, ruler=10.0)
slic.iterate(10)  # You can adjust the number of iterations

# Get the number of superpixels
num_superpixels = slic.getNumberOfSuperpixels()

# Get the labels of the superpixels
labels = slic.getLabels()

# Create an image to visualize the superpixels
mask = slic.getLabelContourMask()
image_superpixels = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2RGBA)
image_superpixels[mask > 0] = [0, 255, 0, 255]  # Set contour pixels to green

# Apply k-means clustering on the superpixels
superpixel_centers = []
for i in range(num_superpixels):
    mask = (labels == i).astype(np.uint8)
    nonzero_points = np.column_stack(np.where(mask > 0))
    center = np.mean(nonzero_points, axis=0)
    superpixel_centers.append(center)

superpixel_centers = np.array(superpixel_centers)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(superpixel_centers)

# Assign cluster labels to superpixels
segmentation_result = np.zeros_like(labels, dtype=np.uint8)
for i, label in enumerate(kmeans.labels_):
    segmentation_result[labels == i] = label

# Visualize the segmentation result
segmentation_result_colored = cv2.applyColorMap(segmentation_result * 50, cv2.COLORMAP_JET)
segmentation_result_colored = cv2.cvtColor(segmentation_result_colored, cv2.COLOR_BGR2RGB)

# Display the original image, superpixels, and segmentation result
cv2.imshow('Original Image', cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))
cv2.imshow('Superpixels', image_superpixels)
cv2.imshow('Segmentation Result', segmentation_result_colored)
cv2.waitKey(0)
cv2.destroyAllWindows()
